# Build RAG + LLM Model

In [ ]:
# Run Docker before
from elasticsearch import Elasticsearch
esrag_client = Elasticsearch('http://localhost:9200')

from tqdm.auto import tqdm

import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

<h3> 1- create our search engine consedering questions, text and section as information and the course as a key field</h3>
<h4>Cette étape s'apelle Keyword search parce que on passe notre query directement dans la dearch function (voir elastic_searc)</h4>

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-ragllm"
esrag_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-ragllm'})

In [12]:
for doc in tqdm(documents):
    esrag_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:06<00:00, 143.37it/s]


In [13]:
def elastic_search(query):
    search_query = {
        "size": 5, # 5 answers in results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],# question field is 3 time more important thant text and section fields
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = esrag_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

<h3>2- After creat awer KB and our search engine, now is time to pass that to a LLM Model</h3>

<h5>2.1- Create prompt (imput od a LLM model) function</h5>

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    QUESTION: {question}
    CONTEXT:  {context} 
    """.strip()
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

<h5>2.2 Create the LLM function</h5>

In [20]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

<h3>3- Time to play the game :)</h3>

In [21]:
def ragllm(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
query = 'I just disovered the course. Can I still join it?'
answer = ragllm(query)
print(answer)

Yes, you can still join the course even after it has started. You don't need to register to be eligible to submit the homework, but be aware of the deadlines for final projects, as it's important not to leave things until the last minute.
